# Debug QubicSkySim

Editing by Louise, March 2020

JCh noticed a very puzzling bug. We think it comes from the new library QubicSkySim.   

In [ ]:
%matplotlib inline
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

# Specific qubic modules
from pysimulators import FitsArray
import pysm
import qubic
from qubic import QubicSkySim as qss
from qubic import NamasterLib as nam

rc('figure', figsize=(12, 8))
rc('font', size=15)

In [ ]:
#### Create sample CMB I,Q,U maps

# dictfilename = 'BmodesNoDustNoSystPaper0_2020.dict'
dictfilename = 'test_QubicSkySim.dict'

# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

d['nside'] = 256
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(center)

The problem is only when nf_sub = 1, as you can try below:

In [ ]:
d['nf_sub'] = 1
lmax = 2 * d['nside']

# Create a sky with qss
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, d)
x0 = Qubic_sky.get_simple_sky_map()
print('x0 Shape:', x0.shape)

# Input map used by qss
inmap = Qubic_sky.input_cmb_maps
print('input Shape:', inmap.shape)

# Anafast spectrum from input map
clsin = hp.anafast(inmap, lmax=lmax)
ll = np.arange(0, lmax+1)
print('ll shape', ll.shape)
# print(ll)
Dlsin = ll * (ll + 1) * clsin / (2 * np.pi)
print('Dlsin', Dlsin.shape)

# Spectra used in qss to create the x0 map
Dlth = Qubic_sky.input_cmb_spectra
print('Dlth', Dlth.shape)
llth = np.arange(len(Dlth[:, 0]))
print('llth shape', llth.shape)
# print(llth)

# Spectra with Anafast from x0 map
Dls = []
for i in range(d['nf_sub']):
    Cls = hp.anafast(x0[i, :, :].T, lmax=lmax)
    Dls.append(ll * (ll + 1) * Cls / (2 * np.pi))
Dls = np.array(Dls)
print('Dls', Dls.shape)
   
    
order = ['TT', 'EE', 'BB', 'TE']
for i in range(4):
    subplot(2, 2, i+1)
    title(order[i])
    xlim(0, lmax)
    plot(llth, Dlth[:, i], 'k', label='Dl Th')
    plot(ll, Dlsin[i, :], alpha=0.3, label='Dl from Input')
    for k in range(d['nf_sub']):
        plot(ll, Dls[k, i, :], alpha=0.3, label='Dl from x0 band {}'.format(k))
legend()